<a href="https://colab.research.google.com/github/pandringa/comp-590-mangroves/blob/master/notebooks/AIPlatformModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/AI_platform_demo.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/AI_platform_demo.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Introduction

This notebook, based on an EarthEngine tutorial one, uses [Google AI Platform](https://cloud.google.com/ml-engine/docs/tensorflow/) to train the model mangrove model for larger datasets (~100k samples) and more epochs (approximately 1 hour per epoch, tested up to 100+).

It has been slightly modified from the original to support multi-core training, along with a few other hyperparameter tweaks. For the original examples, see the links above.

This notebook assumes you have previously exported training data in `.tfrecords` into Google Cloud. If you haven't, you should go to that notebook and do so first.

# Setup software libraries

Install needed libraries to the notebook VM.  Authenticate as necessary.

In [0]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [0]:
# Import and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=YWaYDwiEwI4K7Orx8M8xax7Qcs1C2THAsvyKMQxsUh8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/zAFrs_pLMBf8L7ivuP6YteblhJVmD3w2MyvJAT4v76B1NWHMDvRR598

Successfully saved authorization token.


In [0]:
%tensorflow_version 1.x
# Tensorflow setup.
import tensorflow as tf

# tf.enable_eager_execution()
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [0]:
# Folium setup.
import folium
print(folium.__version__)

0.8.3


# Training code package setup

It's necessary to create a Python package to hold the training code.  Here we're going to get started with that by creating a folder for the package and adding an empty `__init__.py` file.

In [0]:
PACKAGE_PATH = 'ai_platform_fcnn'

!ls -l
!mkdir {PACKAGE_PATH}
!touch {PACKAGE_PATH}/__init__.py
!ls -l {PACKAGE_PATH}

total 8
-rw-r--r-- 1 root root 2659 Apr 27 16:49 adc.json
drwxr-xr-x 1 root root 4096 Apr  3 16:24 sample_data
total 0
-rw-r--r-- 1 root root 0 Apr 27 16:53 __init__.py


## Variables

These variables need to be stored in a place where other code can access them.  There are a variety of ways of accomplishing that, but here we'll use the `%%writefile` command to write the contents of the code cell to a file called `config.py`.

**Note:** You need to insert the name of a bucket (below) to which you have write access!

In [0]:
%%writefile {PACKAGE_PATH}/config.py

import tensorflow as tf

# INSERT YOUR BUCKET HERE!
BUCKET = 'mangroves_model'

# Specify names of output locations in Cloud Storage.
FOLDER = 'fcnn-model-3'
SAVED_JOB_DIR = 'gs://' + BUCKET + '/' + FOLDER + '/trainer_extended'
JOB_DIR = 'gs://' + BUCKET + '/' + FOLDER + '/trainer_balanced'
MODEL_DIR = JOB_DIR + '/model'
LOGS_DIR = JOB_DIR + '/logs'

SAVED_MODEL_DIR = SAVED_JOB_DIR + '/model'

# Pre-computed training and eval data.
DATA_BUCKET = 'mangroves_model'
DATA_FOLDER = 'fcnn-caribbean-mangroves/data-model-3'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
thermalBands = [] # Not on Landsat 7
BANDS = opticalBands + thermalBands
RESPONSE = 'constant'
FEATURES = BANDS + [RESPONSE]
RESOLUTION = 30

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 120000
EVAL_SIZE = 40000

# Specify model training parameters.
BATCH_SIZE = 64
EPOCHS = 10
BUFFER_SIZE = 3000
OPTIMIZER = 'SGD'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']

NUM_CORES = 4

Writing ai_platform_fcnn/config.py


Verify that the written file has the expected contents and is working as intended.

In [0]:
!cat {PACKAGE_PATH}/config.py

import importlib
from ai_platform_fcnn import config
importlib.reload(config)

print('\n\n', config.BATCH_SIZE)


import tensorflow as tf

# INSERT YOUR BUCKET HERE!
BUCKET = 'mangroves_model'

# Specify names of output locations in Cloud Storage.
FOLDER = 'fcnn-model-3'
SAVED_JOB_DIR = 'gs://' + BUCKET + '/' + FOLDER + '/trainer_extended'
JOB_DIR = 'gs://' + BUCKET + '/' + FOLDER + '/trainer_balanced'
MODEL_DIR = JOB_DIR + '/model'
LOGS_DIR = JOB_DIR + '/logs'

SAVED_MODEL_DIR = SAVED_JOB_DIR + '/model'

# Pre-computed training and eval data.
DATA_BUCKET = 'mangroves_model'
DATA_FOLDER = 'fcnn-caribbean-mangroves/data-model-3'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
thermalBands = [] # Not on Landsat 7
BANDS = opticalBands + thermalBands
RESPONSE = 'constant'
FEATURES = BANDS + [RESPONSE]
RESOLUTION = 30

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.i

## Training data, evaluation data and model

The following is code to load training/evaluation data and the model.  Write this into `model.py`. Note that these functions are developed and explained in [this example notebook](https://colab.sandbox.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb).

In [0]:
%%writefile {PACKAGE_PATH}/model.py

from . import config
import tensorflow as tf
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import metrics
from tensorflow.python.keras import models
from tensorflow.python.keras import optimizers

# Dataset loading functions

def parse_tfrecord(example_proto):
  return tf.io.parse_single_example(example_proto, config.FEATURES_DICT)

def to_tuple(inputs):
  inputsList = [inputs.get(key) for key in config.FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(config.BANDS)], stacked[:,:,len(config.BANDS):]

def get_dataset(pattern):
	glob = tf.io.gfile.glob(pattern)
	dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
	dataset = dataset.map(parse_tfrecord)
	dataset = dataset.map(to_tuple)
	return dataset

def get_training_dataset():
	glob = 'gs://' + config.DATA_BUCKET + '/' + config.DATA_FOLDER + '/' + config.TRAINING_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(config.BUFFER_SIZE).batch(config.BATCH_SIZE).repeat()
	return dataset

def get_eval_dataset():
	glob = 'gs://' + config.DATA_BUCKET + '/' + config.DATA_FOLDER + '/' + config.EVAL_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(int(config.BUFFER_SIZE/2)).batch(config.BATCH_SIZE).repeat()
	return dataset

# A variant of the UNET model.

def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = layers.Input(shape=[None, None, len(config.BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(config.OPTIMIZER), 
		loss=losses.get(config.LOSS),
		metrics=[metrics.get(metric) for metric in config.METRICS])

	return model

def get_old_model():
	model = tf.contrib.saved_model.load_keras_model(config.SAVED_MODEL_DIR)
 
	model.compile(
		optimizer=optimizers.get(config.OPTIMIZER), 
		loss=losses.get(config.LOSS),
		metrics=[metrics.get(metric) for metric in config.METRICS])

	return model

Overwriting ai_platform_fcnn/model.py


Verify that `model.py` is functioning as intended.

In [0]:
import importlib
from ai_platform_fcnn import model
importlib.reload(model)

# eval = model.get_eval_dataset()
# print(iter(eval.take(1)).next())

# model = model.get_model()
model = model.get_old_model()
print(model.summary())

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
________________________________________________

## Training task

At this stage, there should be `config.py` storing variables and `model.py` which has code for getting the training/evaluation data and the model.  All that's left is code for training the model.  The following will create `task.py`, which will get the training and eval data, train the model and save it when it's done in a Cloud Storage bucket.

In [0]:
%%writefile {PACKAGE_PATH}/task.py

from . import config
from . import model
import tensorflow as tf

_version_ = "1.3.0"

if __name__ == '__main__':

  training = model.get_training_dataset()
  evaluation = model.get_eval_dataset()


  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
      # call .get_model() if you want to train from scratch
      # or .get_old_model() to train on an existing model (in the OLD_MODEL variable)
      m = model.get_old_model()

    m.fit(
        x=training,
        epochs=config.EPOCHS, 
        steps_per_epoch=int(config.TRAIN_SIZE / config.BATCH_SIZE), 
        validation_data=evaluation,
        validation_steps=int(config.EVAL_SIZE / config.BATCH_SIZE),
        callbacks=[tf.keras.callbacks.TensorBoard(config.LOGS_DIR)])

    tf.contrib.saved_model.save_keras_model(m, config.MODEL_DIR)

Overwriting ai_platform_fcnn/task.py


In [0]:
import importlib
from ai_platform_fcnn import task
importlib.reload(task)

print(task._version_)

1.3.0


# Submit the package to AI Platform for training

Now there's everything to submit this job, which can be done from the command line.  First, define some needed variables.

**Note:** You need to insert the name of a Cloud project (below) you own!

In [0]:
import time

# INSERT YOUR PROJECT HERE!
PROJECT = 'mangrove-ml'

JOB_NAME = 'model3_training_job_cont_' + str(int(time.time()))
TRAINER_PACKAGE_PATH = 'ai_platform_fcnn'
MAIN_TRAINER_MODULE = 'ai_platform_fcnn.task'
REGION = 'us-east1'

Now the training job is ready to be started.  First, you need to enable the ML API for your project.  This can be done from [this link to the Cloud Console](https://console.developers.google.com/apis/library/ml.googleapis.com).  See [this guide](https://cloud.google.com/ml-engine/docs/tensorflow/training-jobs) for details.  Note that the Python and Tensorflow versions should match what is used in the Colab notebook.

In [0]:
!gcloud ai-platform jobs submit training {JOB_NAME} \
    --job-dir {config.JOB_DIR}  \
    --package-path {TRAINER_PACKAGE_PATH} \
    --module-name {MAIN_TRAINER_MODULE} \
    --region {REGION} \
    --project {PROJECT} \
    --runtime-version 1.14 \
    --python-version 3.5 \
    --scale-tier CUSTOM \
    --master-machine-type complex_model_m_gpu

Traceback (most recent call last):
  File "/tools/google-cloud-sdk/lib/gcloud.py", line 95, in <module>
    main()
  File "/tools/google-cloud-sdk/lib/gcloud.py", line 67, in main
    gcloud_main = _import_gcloud_main()
  File "/tools/google-cloud-sdk/lib/gcloud.py", line 48, in _import_gcloud_main
    import googlecloudsdk.gcloud_main
  File "/tools/google-cloud-sdk/lib/googlecloudsdk/gcloud_main.py", line 35, in <module>
    from googlecloudsdk.calliope import cli
  File "/tools/google-cloud-sdk/lib/googlecloudsdk/calliope/cli.py", line 31, in <module>
    from googlecloudsdk.calliope import actions
  File "/tools/google-cloud-sdk/lib/googlecloudsdk/calliope/actions.py", line 29, in <module>
    from googlecloudsdk.calliope import markdown
  File "/tools/google-cloud-sdk/lib/googlecloudsdk/calliope/markdown.py", line 28, in <module>
    from googlecloudsdk.calliope import usage_text
  File "/tools/google-cloud-sdk/lib/googlecloudsdk/calliope/usage_text.py", line 16, in <module>
    "

## Monitor the training job

There's not much more to do until the model is finished training (~24 hours), but it's fun and useful to monitor its progress. You can do that progamatically with another `gcloud` command.  The output of that command can be read into an `IPython.utils.text.SList` from which the `state` is extracted and ensured to be `SUCCEEDED`.  Or you can monitor it from the [AI Platform jobs page](http://console.cloud.google.com/ai-platform/jobs) on the Cloud Console.

In [0]:
desc = !gcloud ai-platform jobs describe {JOB_NAME} --project {PROJECT}
state = desc.grep('state:')[0].split(':')[1].strip()
print(state)

# Inspect the trained model

Once the training job has finished, verify that you can load the trained model and print a summary of the fitted parameters.  It's also useful to examine the logs with [TensorBoard](https://www.tensorflow.org/guide/summaries_and_tensorboard).  There's a convenient notebook extension that will launch TensorBoard in the Colab notebook.  Examine the training and testing learning curves to ensure that the training process has converged.

In [0]:
%load_ext tensorboard
%tensorboard --logdir {config.LOGS_DIR}

# Prepare the model for making predictions in Earth Engine

Before we can use the model in Earth Engine, it needs to be hosted by AI Platform.  But before we can host the model on AI Platform we need to *EEify* (a new word!) it.  The EEification process merely appends some extra operations to the input and outputs of the model in order to accomdate the interchange format between pixels from Earth Engine (float32) and inputs to AI Platform (base64).  (See [this doc](https://cloud.google.com/ml-engine/docs/online-predict#binary_data_in_prediction_input) for details.)  

## `earthengine model prepare`
The EEification process is handled for you using the Earth Engine command `earthengine model prepare`.  To use that command, we need to specify the input and output model directories and the name of the input and output nodes in the TensorFlow computation graph.  We can do all that programmatically:

In [0]:
from ai_platform_fcnn import config
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(config.MODEL_DIR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to 
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "impervious"}) + "'"

# Put the EEified model next to the trained model directory.
EEIFIED_DIR = config.JOB_DIR + '/eeified'

# You need to set the project before using the model prepare command.
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {config.MODEL_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

Successfully saved project id
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

Success: model at 'gs://mangroves_model/fcnn-model-3/trainer_extended/eeified' is ready to be hosted in AI Platform.


Note that you can also use the TensorFlow saved model command line tool to do this manually.  See [this doc](https://www.tensorflow.org/guide/saved_model#cli_to_inspect_and_execute_savedmodel) for details.  Also note the names we've specified for the new inputs and outputs: `array` and `impervious`, respectively.

# Perform inference using the trained model in Earth Engine

Before it's possible to get predictions from the trained and EEified model, it needs to be deployed on AI Platform.  The first step is to create the model.  The second step is to create a version.  See [this guide](https://cloud.google.com/ml-engine/docs/tensorflow/deploying-models) for details.  Note that models and versions can be monitored from the [AI Platform models page](http://console.cloud.google.com/ai-platform/models) of the Cloud Console. 

To ensure that the model is ready for predictions without having to warm up nodes, you can use a configuration yaml file to set the scaling type of this version to autoScaling, and, set a minimum number of nodes for the version. This will ensure there are always nodes on stand-by, however, you will be charged as long as they are running. For this example, we'll set the minNodes to 10. That means that at a minimum, 10 nodes are always up and running and waiting for predictions. The number of nodes will also scale up automatically if needed.

In [0]:
%%writefile config.yaml
autoScaling:
    minNodes: 1

Overwriting config.yaml


In [0]:
from ai_platform_fcnn import config

MODEL_NAME = 'mangroves_predictor'
VERSION_NAME = 'v1'
print('Creating version: ' + VERSION_NAME)

!gcloud ai-platform models create {MODEL_NAME} --region {REGION} --project {PROJECT}
!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --runtime-version=1.14 \
  --framework "TENSORFLOW" \
  --python-version=3.5 \
  --config=config.yaml

Creating version: v1
ERROR: (gcloud.ai-platform.models.create) Project [mangrove-ml] not found: <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/goo

KeyboardInterrupt: ignored

There is now a trained model, prepared for serving to Earth Engine, hosted and versioned on AI Platform.  We can now connect Earth Engine directly to the trained model for inference.  You do that with the `ee.Model.fromAiPlatformPredictor` command.

## `ee.Model.fromAiPlatformPredictor`
For this command to work, we need to know a lot about the model.  To connect to the model, you need to know the name and version.

### Inputs
You need to be able to recreate the imagery on which it was trained in order to perform inference.  Specifically, you need to create an array-valued input from the scaled data and use that for input.  (Recall that the new input node is named `array`, which is convenient because the array image has one band, named `array` by default.)  The inputs will be provided as 144x144 patches (`inputTileSize`), at 30-meter resolution (`proj`), but 8 pixels will be thrown out (`inputOverlapSize`) to minimize boundary effects.

### Outputs
The output (which you also need to know), is a single float band named `impervious`.

In [0]:
from ai_platform_fcnn import config

# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')

# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  mask2 = image.mask().reduce('min')
  mask3 = image.select(config.opticalBands).gt(0).And(
          image.select(config.opticalBands).lt(10000)).reduce('min')
  mask = mask1.And(mask2).And(mask3)
  return image.select(config.opticalBands).divide(10000).updateMask(mask)

# The image input data is a cloud-masked median composite.
YEAR = 2019
print("Year is "+str(YEAR))

CARIBBEAN_GEO = ee.Geometry.Polygon(
         [[-117, 32],
          [-117, 0],
          [-53, 0],
          [-53, 32]]);

possible_mangroves = (ee.FeatureCollection("users/pandringa/gmw_2016")
  .filterBounds(CARIBBEAN_GEO)
  .map(lambda f: f.buffer(1000)))

ee.batch.Export.table.toAsset(
    possible_mangroves,
    description="caribbean_mangroves_union",
    assetId="users/pandringa/caribbean_mangroves_union_2016"
).start()

image = l8sr.filterDate(str(YEAR)+'-01-01', str(YEAR)+'-12-31').map(maskL8sr).median()
image = image.clip(possible_mangroves)

mapid = image.getMapId({'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 0.3})
map = folium.Map()
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map


# Load the trained model and use it for prediction.
predictor = ee.Model.fromAiPlatformPredictor(
    projectName = PROJECT,
    modelName = MODEL_NAME,
    version = VERSION_NAME,
    inputTileSize = [144, 144],
    inputOverlapSize = [8, 8],
    proj = ee.Projection('EPSG:4326').atScale(30),
    fixInputProj = True,
    outputBands = {'impervious': {
        'type': ee.PixelType.float()
      }
    }
)
predictions = predictor.predictImage(image.toFloat().toArray())

# Mask image to regions we expect mangroves to be near
predictions = predictions.mask(ee.Image(0).byte().paint(possible_mangroves, 1))

ee.batch.Export.image.toAsset(
    image=predictions,
    assetId="users/pandringa/final_model_predictions",
    description="model3_caribbean_predictions",
    region=possible_mangroves,
    scale=30,
    maxPixels=1e13
).start()

# Use folium to visualize the input imagery and the predictions.
MODEL_TEST_LOC = [8.527453749657104,-83.29948528935525]

map_id_dict = image.getMapId({'bands': ['B1', 'B2', 'B3'], 'min': 0, 'max': 0.3})
map = folium.Map(location=MODEL_TEST_LOC, zoom_start=13)
folium.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)

map_id_dict = predictions.getMapId({'min': 0, 'max': 1})

folium.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='predictions',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Year is 2019
